In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
data_dir = 'caltech101/101_ObjectCategories'

In [3]:
image_size = (224, 224)
batch_size = 32

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.15
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  
)

In [ ]:
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False)


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)  
predictions = Dense(102, activation='softmax')(x)

In [ ]:
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
epochs = 25
h=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[reduce_lr, early_stopping]  
)

In [ ]:

from tensorflow.keras.preprocessing.image import load_img
image = load_img('caltech101/101_ObjectCategories/BACKGROUND_Google/image_0016.jpg', target_size=(224, 224)) 

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
image = img_to_array(image)

In [ ]:
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input
image = preprocess_input(image)

In [ ]:
import numpy as np
pred = model.predict(image) 
predicted_class_index = np.argmax(pred,axis=-1)
class_labels = list(train_generator.class_indices.keys())
predicted_class_label = class_labels[predicted_class_index[0]]

print(predicted_class_label)

In [ ]:
print(pred)

In [ ]:
print(list(train_generator.class_indices.keys()))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(h.history['accuracy']) #get the accuracy for the training set from the model's history attribute
plt.plot(h.history['val_accuracy'])   #get the loss for the training set from the model's history attribute
plt.title('Training Loss and Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'])

In [ ]:
plt.plot(h.history['loss']) #get the accuracy for the training set from the model's history attribute
plt.plot(h.history['val_loss'])   #get the loss for the training set from the model's history attribute
plt.title('Training Loss and Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'])